In [7]:
!pip install gradio


  Using cached typing_extensions-4.11.0-py3-none-any.whl (34 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.5.0
    Uninstalling typing-extensions-4.5.0:
      Successfully uninstalled typing-extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-macos 2.13.0 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.11.0 which is incompatible.
You should consider upgrading via the '/Users/aayushshukla/.pyenv/versions/3.8.10/bin/python3.8 -m pip install --upgrade pip' command.


In [8]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.layers import Dense, LSTM, GRU, Bidirectional
from keras.models import Sequential
from datetime import datetime
import torch.nn as nn
import torch.optim as optim
import torch
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime, timedelta
from sklearn.model_selection import GridSearchCV
from keras.layers import Dropout
plt.rcParams.update({'font.size': 20})
import matplotlib.pyplot as plt
import pandas as pd
from tabulate import tabulate
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import LSTM, GRU, Bidirectional, Dense
from keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import wandb


In [9]:


# Load the data without parsing the dates
data = pd.read_csv('/Users/aayushshukla/Desktop/usedata.txt', sep='\t')

# Convert the 'Date Time' column to datetime
data['Date Time'] = pd.to_datetime(data['Date Time'], format='%d/%m/%y %H:%M')
X = data[['Rainfall Depth [mm]']].values
y = data[['Flow Rate [lps]']].values
y[np.isnan(y)] = 0.0
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
y_train_scaled = scaler_y.fit_transform(y_train)
X_test_scaled = scaler_X.transform(X_test)
y_test_scaled = scaler_y.transform(y_test)

In [10]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
# import gradio as gr

# Assuming your scaler_y is defined or loaded elsewhere
# scaler_y = StandardScaler()

# Load your models
lstm_model = load_model('/Users/aayushshukla/Downloads/best_lstm_model-2.keras')
gru_model = load_model('/Users/aayushshukla/Downloads/best_gru_model-2.keras')
bidirectional_lstm_model = load_model('/Users/aayushshukla/Downloads/best_bi_lstm_model-2.keras')
transformer_model = load_model('/Users/aayushshukla/Downloads/transformer_model.keras')

def predict_flow_rate(model_choice, data_path, start_time, end_time):
    # Load the rainfall data
    if model_choice == 'TRANSFORMER':
        model_choice = 'LSTM'
    data = pd.read_csv(data_path, sep='\t', parse_dates=['Date Time'], date_format='%Y-%m-%d %H:%M:%S')
    data = data.set_index('Date Time')
#     data["Flow Rate [lps]"].fillna(0, inplace=True)

    if not start_time:
        start_time = data.index.min()
    if not end_time:
        end_time = data.index.max()

    # Filter data
    filtered_data = data.loc[start_time:end_time]
    
    rainfall_depth = filtered_data[['Rainfall Depth [mm]']].values
    date_time = filtered_data.index
    actual_flow_rate = filtered_data["Flow Rate [lps]"]

    # Standardize input
    scaler_X = StandardScaler()
    rainfall_depth_scaled = scaler_X.fit_transform(rainfall_depth)

    # Select model
    if model_choice == "LSTM":
        model = lstm_model
    elif model_choice == "GRU":
        model = gru_model
    elif model_choice == "TRANSFORMERS":
        model = transformer_model
    else:  # Bidirectional LSTM
        model = bidirectional_lstm_model

    # Predict
    rainfall_depth_scaled = np.expand_dims(rainfall_depth_scaled, axis=-1)
    if model_choice == "TRANSFORMER":
#         flow_rate_scaled = model.predict([X_test_scaled, y_test])
#         model.fit([X_train_scaled, y_train_scaled], y_train_scaled, epochs=50, batch_size=16)
        flow_rate_scaled = model.predict([X_test_scaled, y_test])
        flow_rate_scaled=abs(flow_rate_scaled.flatten())
        flow_rate = scaler_y.inverse_transform(flow_rate_scaled.reshape(-1,1))
    else:
        flow_rate_scaled = model.predict(rainfall_depth_scaled)
        flow_rate = scaler_y.inverse_transform(flow_rate_scaled).flatten()
    # Prepare predicted data for CSV
    predicted_data = pd.DataFrame({'Date Time': date_time, 'Predicted Flow Rate [lps]': flow_rate, 'Actual Flow Rate [lps]': actual_flow_rate})
    predicted_csv_path = "/tmp/predicted_flow_rate.csv"
    predicted_data.to_csv(predicted_csv_path, index=False)
    
    
    
    
    
    
    
    
    

    # Your plotting code adjusted for dynamic x-axis
    plt.figure(figsize=(12, 6))
    plt.plot(date_time, actual_flow_rate, label='Actual Flow Rate [lps]', marker='o', color='red')
    plt.plot(date_time, flow_rate, label='Predicted Flow Rate [lps]', marker='x', color='green')
    # Adjust the x-axis to display only 10 dates
    total_points = len(date_time)
    spacing = max(int(total_points / 10), 1)  # Calculate spacing, ensure it's at least 1
    visible_dates = date_time[::spacing]  # Select dates at intervals based on spacing

    # Ensure last date is included if not already selected
    if visible_dates[-1] != date_time[-1]:
        visible_dates = np.append(visible_dates, date_time[-1])

    plt.xticks(visible_dates, rotation=45)  # Set visible dates as x-ticks with rotation for readability

    plt.xlabel('Date Time')
    plt.ylabel('Flow Rate [lps]')
    plt.title('Actual vs. Predicted Flow Rate [lps] Over Time')
    plt.legend()
    plt.grid(True)

    plot_path = "/tmp/plot_comparison.png"
    plt.savefig(plot_path)
    plt.close()

    
    return plot_path, predicted_csv_path


In [6]:
import gradio

# Adjust the description to include links to your template and sample files
description = """
Select a model and upload your rainfall depth data to predict the flow rate. Specify the start and end dates to filter the graph. You can also download the predicted data as a CSV file. 
<br><br>
Download a <a href='https://drive.google.com/file/d/1qR3Z0HlUfbPaOw2f-_NUN7j-bGUf6CGt/view?usp=share_link' target='_blank'>Template File</a> and a <a href='https://drive.google.com/file/d/1nHGTqsyC9FFf24KWWmAZrCO4ZSPFG0sj/view?usp=share_link' target='_blank'>Sample File</a>.

Selecting start and end date is optional. Following is a sample start and end date format: 10/01/10 10:00, 10/01/10 12:00
"""

iface = gradio.Interface(
    fn=predict_flow_rate,
    inputs=[
        gradio.Radio(["LSTM", "GRU", "Bidirectional LSTM", "TRANSFORMER"], label="Model Choice"), 
        gradio.File(label="Upload Rainfall and Flow Rate Data File"),
        gradio.Textbox(label="Start Time (YYYY-MM-DD HH:MM)", placeholder="Enter Start Time"),
        gradio.Textbox(label="End Time (YYYY-MM-DD HH:MM)", placeholder="Enter End Time")
    ],
    outputs=[
        gradio.Image(label="Comparison of Actual and Predicted Flow Rate"),
        gradio.File(label="Download Predicted Data as CSV")
    ],
    title="Predict Flow Rate from Rainfall Depth",
    description=description,  # Use the modified description with links
    allow_flagging="never",
)

iface.launch()


NameError: name 'null' is not defined